Spam messages refer to unsolicited or unwanted messages/emails that are sent in bulk to users. In most messaging/emailing services, messages are detected as spam automatically so that these messages do not unnecessarily flood the users’ inboxes. These messages are usually promotional and peculiar in nature. Thus, it is possible for us to build ML/DL models that can detect Spam messages.

we’ll build a TensorFlow-based Spam detector; in simpler terms, we will have to classify the texts as Spam or Ham.Spam detection is a case of a Text Classification problem.
we’ll be performing EDA on our dataset and building a text classification model.

In [3]:
# Importing necessary libraries for EDA
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
 
import string
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
nltk.download('stopwords')
 
# Importing libraries necessary for Model Building and Training
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
 
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


In [4]:
import os
data_path = "/kaggle/input/"
print(os.listdir(data_path))

['bank-transaction-fraud-detection', 'spam-email-classification']


<!-- Spam messages refer to unsolicited or unwanted messages/emails that are sent in bulk to users. In most messaging/emailing services, messages are detected as spam automatically so that these messages do not unnecessarily flood the users’ inboxes. These messages are usually promotional and peculiar in nature. Thus, it is possible for us to build ML/DL models that can detect Spam messages. -->

In [5]:
data = pd.read_csv('/kaggle/input/spam-email-classification/email.csv')

In [6]:
data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [7]:
data.shape

(5573, 2)

In [8]:
# Rename columns for better consistency and clarity
data.rename(columns={'Category': 'Label', 'Message': 'Message'}, inplace=True)

# Convert 'Label' column values to numerical values (ham = 0, spam = 1)
data['Label'] = data['Label'].map({'ham': 0, 'spam': 1})

# Separate features (X) and target labels (y)
X = data['Message']
y = data['Label']

In [9]:
print(data['Label'].isnull().sum(), "missing value")

# Fill any missing values in the 'Label' column with 0 (assuming 'ham' as the default)
data['Label'] = data['Label'].fillna(0)

# Recheck for missing values in the 'Label' column after filling and print the result
print(data['Label'].isnull().sum(), "missing value")

1 missing value
0 missing value


In [10]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets (80% training, 20% testing)
X = data['Message']
y = data['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Convert the text data into numerical features again with n-grams (1 to 2-grams)
tfidf = TfidfVectorizer(stop_words='english', max_features=2000, ngram_range=(1, 2))
X_train_vec = tfidf.fit_transform(X_train) # Fit and transform the training data
X_test_vec = tfidf.transform(X_test) # Transform the testing data based on the fitted model

# Check the shape of the transformed training data and the length of the training labels
print(X_train_vec.shape)
print(len(y_train))

(4458, 2000)
4458


In [12]:
# Align the training features (X_train) and labels (y_train) by matching indices (keeping only the common ones)
X_train, y_train = X_train.align(y_train, join='inner')

from sklearn.naive_bayes import MultinomialNB

# Create and train a Naive Bayes model
model = MultinomialNB()
model.fit(X_train_vec, y_train)

MultinomialNB()

In [13]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Predict the labels for the test set
y_pred = model.predict(X_test_vec)

# Print the accuracy of the model
print("Accuracy:", accuracy_score(y_test, y_pred))

# Print the classification report (precision, recall, f1-score)
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Display the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:\n", conf_matrix)

Accuracy: 0.979372197309417

Classification Report:
               precision    recall  f1-score   support

         0.0       0.98      1.00      0.99       958
         1.0       0.99      0.86      0.92       157

    accuracy                           0.98      1115
   macro avg       0.99      0.93      0.95      1115
weighted avg       0.98      0.98      0.98      1115


Confusion Matrix:
 [[957   1]
 [ 22 135]]
